In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import random
import battery_sim

bat = battery_sim.battery()

In [3]:
# Get bounds for operation - charge and discharge:
print(f"Available Energy {bat.avl_energy} kWh")
print(f"Max input: {bat.get_charge_potential()} kWh")
print(f"Max output: {bat.get_discharge_potential()} kWh")

Available Energy 0 kWh
Max input: 0.41666666666666663 kWh
Max output: 0.0 kWh


In [4]:
# Charge the battery with too much power
bat.charge_(11) 

BatteryOverflowError: 

In [5]:
# Charge the battery with max charge rate
first_charge = bat.get_charge_potential() 

# charge battery
bat.charge_(first_charge)
# print updated available energy
bat.avl_energy

0.3952847075210474

In [6]:
# Check that new availabe energy = max charge rate x charge losses
bat.avl_energy == bat.get_charge_potential() * bat.charge_loss

True

In [7]:
# Get discharge potential - how much energy i could recover, taking into account discharge losses
first_discharge = bat.get_discharge_potential()
first_discharge

0.37499999999999994

In [8]:
# Check round trip efficiency calculation:
first_discharge / first_charge

0.8999999999999999

In [9]:
# Discharge battery
bat.discharge_(first_discharge)

(0.3952847075210474, 0.0)

In [10]:
# Check empty.
bat.avl_energy

0.0

In [11]:
# degredation demo
t = battery_sim.battery()

# Random array of outputs
energy= [-5,3,2,5,-10,3,5,8,10,2,-4,6,-3,2,1,3,5,2,9]

for e in energy:
    # decrememnt only (doesn't call the charging/discharing functions)
    t.update_capacity_degredation(e)
    print(f"Max Capacity: {t.capacity:.5f}, Total Energy in/out: {t.total_energy_flow}, Energy:{e}, trips: {t.trip_count}, previous: {t.previous_trip_count}")

Max Capacity: 14.00000, Total Energy in/out: 5, Energy:-5, trips: 0, previous: 0
Max Capacity: 14.00000, Total Energy in/out: 8, Energy:3, trips: 0, previous: 0
Max Capacity: 14.00000, Total Energy in/out: 10, Energy:2, trips: 0, previous: 0
Max Capacity: 14.00000, Total Energy in/out: 15, Energy:5, trips: 0, previous: 0
Max Capacity: 14.00000, Total Energy in/out: 25, Energy:-10, trips: 0, previous: 0
Max Capacity: 13.99975, Total Energy in/out: 28, Energy:3, trips: 1, previous: 1
Max Capacity: 13.99975, Total Energy in/out: 33, Energy:5, trips: 1.0, previous: 1.0
Max Capacity: 13.99975, Total Energy in/out: 41, Energy:8, trips: 1.0, previous: 1.0
Max Capacity: 13.99975, Total Energy in/out: 51, Energy:10, trips: 1.0, previous: 1.0
Max Capacity: 13.99975, Total Energy in/out: 53, Energy:2, trips: 1.0, previous: 1.0
Max Capacity: 13.99950, Total Energy in/out: 57, Energy:-4, trips: 2.0, previous: 2.0
Max Capacity: 13.99950, Total Energy in/out: 63, Energy:6, trips: 2.0, previous: 2.0
M

### Simulation - Status Quo

Interval = 5 minutes

#### Tariff Spects:

- Import Tariff: ToU
- Export Tariff: SolarFit

#### Battery Specs

- Grid Export Allowed: False
- Grid Charging Allowed: False
- Power Priority: Load
- Charge Priority: Solar

#### Solar Specs:

- Solar Pesent: True

#### Sign Conventions:

Note: Solar cannot import, and loads cannot export energy.

| Component | Import | Export |
| --- | --- | --- |
| **Solar** | Not Applicable | Negative (-) |
| **Battery** | (Charge) Positive (+) | (Discharge) Negative (-) |
| **Loads** | (Consumption) Positive (+) | Not Applicable |
| **Grid** | (Consumption) Positive (+) | Negative (-) |


Logic

|          State                               |        Action Space             | Comment |
| --- | --- | --- |
| solar on, with load, solar greater than load | battery: [0, abs(solar + load)] |charging from solar excess   |
| solar on, with load, solar less than load    | battery: [0, abs(solar + load)] |powering deficit load   |
| solar on, no load                            | battery: [0, abs(solar)]        |charging from solar excess     |
| solar off, with load                         | battery: [-load, 0]             |battery can power load  |
| solar off, no load                           | battery: [0]                    |battery can do nothing  |

In [13]:
print("""
Battery cannot export or charge from the grid (solar only). We expect to see:
- charge/discharge values not greater than the limit value. 
- charge only possible with net solar, and bounded appropriately by max or available solar. 
- discharge only possible with available capacity, and bounded by max or available load.
""")

for k in [0, 0.5, 1]:
    
    bat = battery_sim.battery(dischargeable=False, importable=False)
    bat.capacity = 1
    bat.avl_energy =k

    # Get bounds for operation - charge and discharge:
    print(f"\nAvailable Energy {bat.avl_energy} kWh\n"
          f"Max input: {bat.get_charge_potential()} kWh\n"
          f"Max output: {bat.get_discharge_potential()} kWh\n")

    for i in range(20):
        Load = random.random()
        Solar = random.random() * -1

        max_c = bat.get_charge_potential()
        max_d = bat.get_discharge_potential()
        a_d, a_c = bat.get_limits(Solar, Load)
        print(f"Net: {Solar+Load:3.4f}, SoC: {bat.avl_energy / bat.capacity * 100 :2.4f}%"
              f", Discharge (bound/limit): {a_d:2.4f}/{max_d:2.4f}, "
              f"Charge (bound/limit): {a_c:2.4f}/{max_c:2.4f}")


Battery cannot export or charge from the grid (solar only). We expect to see:
- charge/discharge values not greater than the limit value. 
- charge only possible with net solar, and bounded appropriately by max or available solar. 
- discharge only possible with available capacity, and bounded by max or available load.


Available Energy 0 kWh
Max input: 0.41666666666666663 kWh
Max output: 0.0 kWh

Net: 0.4358, SoC: 0.0000%, Discharge (bound/limit): -0.0000/0.0000, Charge (bound/limit): 0.0000/0.4167
Net: -0.3645, SoC: 0.0000%, Discharge (bound/limit): 0.0000/0.0000, Charge (bound/limit): 0.3645/0.4167
Net: 0.1749, SoC: 0.0000%, Discharge (bound/limit): -0.0000/0.0000, Charge (bound/limit): 0.0000/0.4167
Net: 0.6593, SoC: 0.0000%, Discharge (bound/limit): -0.0000/0.0000, Charge (bound/limit): 0.0000/0.4167
Net: 0.2412, SoC: 0.0000%, Discharge (bound/limit): -0.0000/0.0000, Charge (bound/limit): 0.0000/0.4167
Net: 0.0336, SoC: 0.0000%, Discharge (bound/limit): -0.0000/0.0000, Charge (b

In [16]:
print("""
Battery can export, but cannot charge from the grid (solar only). We expect to see:
- charge/discharge values not greater than the limit value. 
- charge only possible with net solar, and bounded appropriately by max or available solar. 
- discharge only possible with available capacity, and bounded by max - can be greater than load!.
""")

for k in [0, 0.4, 1]:
    
    bat = battery_sim.battery(dischargeable=True, importable=False)
    bat.capacity = 1
    bat.avl_energy =k

    # Get bounds for operation - charge and discharge:
    print(f"\nAvailable Energy {bat.avl_energy} kWh\n"
          f"Max input: {bat.get_charge_potential()} kWh\n"
          f"Max output: {bat.get_discharge_potential()} kWh\n")

    for i in range(20):
        Load = random.random()
        Solar = random.random() * -1

        max_c = bat.get_charge_potential()
        max_d = bat.get_discharge_potential()
        a_d, a_c = bat.get_limits(Solar, Load)
        print(f"Net: {Solar+Load:3.4f}, SoC: {bat.avl_energy / bat.capacity * 100 :2.4f}%"
              f", Discharge (bound/limit): {a_d:2.4f}/{max_d:2.4f}, "
              f"Charge (bound/limit): {a_c:2.4f}/{max_c:2.4f}")


Battery cannot export or charge from the grid (solar only). We expect to see:
- charge/discharge values not greater than the limit value. 
- charge only possible with net solar, and bounded appropriately by max or available solar. 
- discharge only possible with available capacity, and bounded by max - can be greater than load!.


Available Energy 0 kWh
Max input: 0.41666666666666663 kWh
Max output: 0.0 kWh

Net: -0.3439, SoC: 0.0000%, Discharge (bound/limit): -0.0000/0.0000, Charge (bound/limit): 0.3439/0.4167
Net: 0.0703, SoC: 0.0000%, Discharge (bound/limit): -0.0000/0.0000, Charge (bound/limit): 0.0000/0.4167
Net: 0.8628, SoC: 0.0000%, Discharge (bound/limit): -0.0000/0.0000, Charge (bound/limit): 0.0000/0.4167
Net: 0.5319, SoC: 0.0000%, Discharge (bound/limit): -0.0000/0.0000, Charge (bound/limit): 0.0000/0.4167
Net: -0.6830, SoC: 0.0000%, Discharge (bound/limit): -0.0000/0.0000, Charge (bound/limit): 0.4167/0.4167
Net: 0.1193, SoC: 0.0000%, Discharge (bound/limit): -0.0000/0.000

In [20]:
print("""
Battery cannot export but can charge from the grid (solar only). We expect to see:
- charge/discharge values not greater than the limit value. 
- charge only possible with net solar or grid, and bounded appropriately by max. 
- discharge only possible with available capacity, and bounded by max or available load.
""")

for k in [0, 0.5, 1]:
    
    bat = battery_sim.battery(dischargeable=False, importable=True)
    bat.capacity = 1
    bat.avl_energy =k

    # Get bounds for operation - charge and discharge:
    print(f"\nAvailable Energy {bat.avl_energy} kWh\n"
          f"Max input: {bat.get_charge_potential()} kWh\n"
          f"Max output: {bat.get_discharge_potential()} kWh\n")

    for i in range(20):
        Load = random.random()
        Solar = random.random() * -1

        max_c = bat.get_charge_potential()
        max_d = bat.get_discharge_potential()
        a_d, a_c = bat.get_limits(Solar, Load)
        print(f"Net: {Solar+Load:3.4f}, SoC: {bat.avl_energy / bat.capacity * 100 :2.4f}%"
              f", Discharge (bound/limit): {a_d:2.4f}/{max_d:2.4f}, "
              f"Charge (bound/limit): {a_c:2.4f}/{max_c:2.4f}")


Battery cannot export but can charge from the grid (solar only). We expect to see:
- charge/discharge values not greater than the limit value. 
- charge only possible with net solar or grid, and bounded appropriately by max. 
- discharge only possible with available capacity, and bounded by max or available load.


Available Energy 0 kWh
Max input: 0.41666666666666663 kWh
Max output: 0.0 kWh

Net: 0.1555, SoC: 0.0000%, Discharge (bound/limit): -0.0000/0.0000, Charge (bound/limit): 0.4167/0.4167
Net: -0.0659, SoC: 0.0000%, Discharge (bound/limit): 0.0000/0.0000, Charge (bound/limit): 0.4167/0.4167
Net: 0.1787, SoC: 0.0000%, Discharge (bound/limit): -0.0000/0.0000, Charge (bound/limit): 0.4167/0.4167
Net: 0.1353, SoC: 0.0000%, Discharge (bound/limit): -0.0000/0.0000, Charge (bound/limit): 0.4167/0.4167
Net: 0.0105, SoC: 0.0000%, Discharge (bound/limit): -0.0000/0.0000, Charge (bound/limit): 0.4167/0.4167
Net: 0.4761, SoC: 0.0000%, Discharge (bound/limit): -0.0000/0.0000, Charge (bound/l

In [21]:
print("""
Battery can export and charge from the grid (solar only). We expect to see:
- charge/discharge values not greater than the limit value. 
- charge only possible with net solar and grid, and bounded appropriately by max. 
- discharge only possible with available capacity, and bounded by max - can be greater than load!.
""")

for k in [0, 0.5, 1]:
    
    bat = battery_sim.battery(dischargeable=True, importable=True)
    bat.capacity = 1
    bat.avl_energy =k

    # Get bounds for operation - charge and discharge:
    print(f"\nAvailable Energy {bat.avl_energy} kWh\n"
          f"Max input: {bat.get_charge_potential()} kWh\n"
          f"Max output: {bat.get_discharge_potential()} kWh\n")

    for i in range(20):
        Load = random.random()
        Solar = random.random() * -1

        max_c = bat.get_charge_potential()
        max_d = bat.get_discharge_potential()
        a_d, a_c = bat.get_limits(Solar, Load)
        print(f"Net: {Solar+Load:3.4f}, SoC: {bat.avl_energy / bat.capacity * 100 :2.4f}%"
              f", Discharge (bound/limit): {a_d:2.4f}/{max_d:2.4f}, "
              f"Charge (bound/limit): {a_c:2.4f}/{max_c:2.4f}")


Battery can export and charge from the grid (solar only). We expect to see:
- charge/discharge values not greater than the limit value. 
- charge only possible with net solar and grid, and bounded appropriately by max. 
- discharge only possible with available capacity, and bounded by max - can be greater than load!.


Available Energy 0 kWh
Max input: 0.41666666666666663 kWh
Max output: 0.0 kWh

Net: 0.8340, SoC: 0.0000%, Discharge (bound/limit): -0.0000/0.0000, Charge (bound/limit): 0.4167/0.4167
Net: 0.4880, SoC: 0.0000%, Discharge (bound/limit): -0.0000/0.0000, Charge (bound/limit): 0.4167/0.4167
Net: -0.7588, SoC: 0.0000%, Discharge (bound/limit): -0.0000/0.0000, Charge (bound/limit): 0.4167/0.4167
Net: -0.8500, SoC: 0.0000%, Discharge (bound/limit): -0.0000/0.0000, Charge (bound/limit): 0.4167/0.4167
Net: -0.7167, SoC: 0.0000%, Discharge (bound/limit): -0.0000/0.0000, Charge (bound/limit): 0.4167/0.4167
Net: -0.4113, SoC: 0.0000%, Discharge (bound/limit): -0.0000/0.0000, Charge 

In [47]:
print("""
Battery can export and charge from the grid (solar only). We expect to see:
- charge/discharge values not greater than the limit value. 
- charge only possible with net solar and grid, and bounded appropriately by max. 
- discharge only possible with available capacity, and bounded by max - can be greater than load!.
""")

for k in [0, 0.5, 1]:
    
    bat = battery_sim.battery(dischargeable=True, importable=True)
    bat.capacity = 1
    bat.avl_energy = k

    # Get bounds for operation - charge and discharge:
    print(f"\nAvailable Energy {bat.avl_energy} kWh\n"
          f"Max input: {bat.get_charge_potential():.4f} kWh\n"
          f"Max output: {bat.get_discharge_potential():.4f} kWh\n")

    for i in range(10):
        Load = random.random()
        Solar = random.random() * -1

        max_c = bat.get_charge_potential()
        max_d = bat.get_discharge_potential()
        a_d, a_c = bat.get_limits(Solar, Load)
        print(f"Net: {Solar+Load:3.4f}, Avl Energy: {bat.avl_energy:2.4f} kWh"
              f", Discharge (bound/limit): {a_d:2.4f}/{max_d:2.4f}, "
              f"Charge (bound/limit): {a_c:2.4f}/{max_c:2.4f}")
        
        energy = random.sample(np.linspace(a_d, a_c, 100).tolist(), 1)[0]
        if energy > 0:
            print(f"Charging! {energy:2.4f} kWh")
            bat.charge_(energy)
        else:
            print(f"Discharging! {energy:2.4f} kWh")
            bat.discharge_(-energy)


Battery can export and charge from the grid (solar only). We expect to see:
- charge/discharge values not greater than the limit value. 
- charge only possible with net solar and grid, and bounded appropriately by max. 
- discharge only possible with available capacity, and bounded by max - can be greater than load!.


Available Energy 0 kWh
Max input: 0.4167 kWh
Max output: 0.0000 kWh

Net: 0.4037, Avl Energy: 0.0000 kWh, Discharge (bound/limit): -0.0000/0.0000, Charge (bound/limit): 0.4167/0.4167
Charging! 0.0547 kWh
Net: -0.2937, Avl Energy: 0.0519 kWh, Discharge (bound/limit): -0.0492/0.0492, Charge (bound/limit): 0.4167/0.4167
Charging! 0.2896 kWh
Net: 0.9793, Avl Energy: 0.3266 kWh, Discharge (bound/limit): -0.3099/0.3099, Charge (bound/limit): 0.4167/0.4167
Discharging! -0.1484 kWh
Net: 0.5653, Avl Energy: 0.1702 kWh, Discharge (bound/limit): -0.1615/0.1615, Charge (bound/limit): 0.4167/0.4167
Charging! 0.2064 kWh
Net: 0.5393, Avl Energy: 0.3660 kWh, Discharge (bound/limit): -0